<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/keras_model_subclassing_dogcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization, Activation

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD

import os
import random
import shutil

In [ ]:
!unzip drive/MyDrive/Dataset/dog_cat.zip

In [4]:
data_dir = "./dog_cat"
dog = []
cat = []
for img in os.listdir(data_dir):
    label = img.split(".")[0]
    img_path = os.path.join(data_dir, img)
    if label == "dog":
        dog.append(img_path)
    else:
        cat.append(img_path)

rate = 0.2
test_cat = random.sample(cat, int(rate*len(cat)))
test_dog = random.sample(dog, int(rate*len(dog)))

train_cat = list(set(cat) ^ set(test_cat))
train_dog = list(set(dog) ^ set(test_dog))

save_dir = "./data/"
for test_cat_img, test_dog_img in zip(test_cat, test_dog):
    img_cat_name = test_cat_img.split("/")[-1]
    img_dog_name = test_dog_img.split("/")[-1]

    save_test_cat = os.path.join(save_dir,"test","cat")
    os.makedirs(save_test_cat, exist_ok=True)
    save_test_dog = os.path.join(save_dir,"test","dog")
    os.makedirs(save_test_dog, exist_ok=True)

    shutil.copy2(test_cat_img, os.path.join(save_test_cat, img_cat_name))
    shutil.copy2(test_dog_img, os.path.join(save_test_dog, img_dog_name))

for train_cat_img, train_dog_img in zip(train_cat, train_dog):
    img_cat_name = train_cat_img.split("/")[-1]
    img_dog_name = train_dog_img.split("/")[-1]

    save_train_cat = os.path.join(save_dir,"train","cat")
    os.makedirs(save_train_cat, exist_ok=True)
    save_train_dog = os.path.join(save_dir,"train","dog")
    os.makedirs(save_train_dog, exist_ok=True)

    shutil.copy2(train_cat_img, os.path.join(save_train_cat, img_cat_name))
    shutil.copy2(train_dog_img, os.path.join(save_train_dog, img_dog_name))

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_set = train_datagen.flow_from_directory('data/train',
                                                target_size=(200, 200),
                                                batch_size=32,
                                                class_mode='binary', shuffle=True)
test_set = test_datagen.flow_from_directory('data/test',
                                                target_size=(200, 200),
                                                batch_size=32,
                                                class_mode='binary',shuffle=False)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = Conv2D(32, (5,5), padding="same")
        self.act1 = Activation("relu")
        self.pool1 = MaxPooling2D(pool_size=(2, 2))
        self.conv2 = Conv2D(48, (5,5), padding="valid")
        self.act2 = Activation("relu")
        self.pool2 = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(256)
        self.act3 = Activation("relu")
        self.dense2 = Dense(84)
        self.act4 = Activation("relu")
        self.dense3 = Dense(1)
        self.act5 = Activation("softmax")
    def call(self, inputs):
        x = self.pool1(self.act1(self.conv1(inputs)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.flatten(x)
        x = self.act3(self.dense1(x))
        x = self.act4(self.dense2(x))
        x = self.act5(self.dense3(x))
        return x

In [12]:
net = LeNet()
net.compile(optimizer=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True),
                    loss="binary_crossentropy",
                    metrics=['accuracy'])
history = net.fit(train_set, steps_per_epoch=(20000//32),epochs= 5,
                             validation_data = test_set, validation_steps= (5000//32),
                            verbose=1)

checkpoint_dir = "./checkpoint/"
os.makedirs(checkpoint_dir, exist_ok=True)

net.save_weights(os.path.join(checkpoint_dir, "keras_cifar10.h5"))

# evaluate model and save
_, acc = net.evaluate(test_set,steps = (5000//32), verbose = 0)
print(f"Accuracy: {acc*100}")

Epoch 1/5
625/625 [==============================] - 268s 428ms/step - loss: 0.6722 - accuracy: 0.5856 - val_loss: 0.6201 - val_accuracy: 0.6581
Epoch 2/5
625/625 [==============================] - 269s 430ms/step - loss: 0.6001 - accuracy: 0.6751 - val_loss: 0.5490 - val_accuracy: 0.7115
Epoch 3/5
625/625 [==============================] - 270s 432ms/step - loss: 0.5232 - accuracy: 0.7402 - val_loss: 0.4768 - val_accuracy: 0.7762
Epoch 4/5
625/625 [==============================] - 269s 431ms/step - loss: 0.4775 - accuracy: 0.7671 - val_loss: 0.4453 - val_accuracy: 0.7927
Epoch 5/5
625/625 [==============================] - 269s 430ms/step - loss: 0.4425 - accuracy: 0.7936 - val_loss: 0.4472 - val_accuracy: 0.7917
Accuracy: 79.16666865348816
